In [20]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# set the max columns to none
pd.set_option('display.max_columns', None)

import tensorflow as tf
from tensorflow import keras

## Data Hotel Goibibo 

In [21]:
df_goibibo = pd.read_csv('goibibo_com-travel_sample.csv')
# select potential parameter only
df_goibibo.drop(columns=['room_type','room_count','additional_info','province','area','locality','crawl_date','hotel_brand','hotel_category','pageurl','qts','query_time_stamp','room_area','similar_hotel','sitename','uniq_id','image_count'], inplace=True)
# drop data review
df_goibibo.drop(columns=['review_count_by_category','site_review_count','site_stay_review_rating','guest_recommendation'], inplace=True)

df_goibibo.head(3)

,address,city,country,hotel_description,hotel_facilities,hotel_star_rating,latitude,longitude,point_of_interest,property_id,property_name,property_type,room_facilities,site_review_rating,state
0,"15th Mile, N.H.21,Manali, District Kullu,Himac...",Manali,India,The standard check-in time is 12:00 PM and the...,Doctor on Call|Dry Cleaning|Laundry Service Av...,2,32.139387,77.154660,Hadimba Temple|Naggar Village|Himalayan Nyingm...,HTLGBO1000016238,Baragarh Regency,Resort,Room Service |Basic Bathroom Amenities|Cable /...,4.0,Himachal Pradesh
1,"A-585, Sushant Lok-1 ,Near Iffco Chowk Metro S...",Gurgaon,India,The standard check-in time is 12:00 PM and the...,Airport Transfer Available / Surcharge|Banquet...,0,28.472097,77.072546,Sushant Lok|Sahara Mall|Amity International Sc...,HTLGBO1000015822,Asian Suites A- 585,Guest House,Room Service |Air Conditioning |Basic Bathroom...,4.5,Haryana
2,"Cobra Vaddo,Calungate Baga Road, Bardez, Calan...",Goa,India,The standard check-in time is 12:00 PM and the...,Swimming Pool|Bar / Lounge |Laundry Service Av...,0,15.548398,73.757634,"Anjuna Beach|Calangute Beach|Titos lane, baga|...",HTLGBO1000004551,Bevvan Resort,Resort,Room Service |Air Conditioning |Cable / Satell...,2.5,Goa


In [24]:
# eksplorasi dataset
print('jumlah property id: ',len(df_goibibo['property_id'].unique()))
print('jumlah property name: ',len(df_goibibo['property_name'].unique()))
print('jumlah property type: ',len(df_goibibo['property_type'].unique()))

df_goibibo.dropna(inplace=True)
df_goibibo.drop_duplicates(subset=['property_name'], inplace=True)
print('jumlah data setelah drop null dan duplicates: ',len(df_goibibo.dropna()))

jumlah property id:  1953
jumlah property name:  1953
jumlah property type:  17
jumlah data setelah drop null dan duplicates:  1953


In [25]:
print('cek jumlah null dari tiap parameter: \n')
print('property_name: ', df_goibibo['property_name'].isna().sum())
print('property_type: ', df_goibibo['property_type'].isna().sum())
print('property_id: ', df_goibibo['property_id'].isna().sum())
print('hotel_description: ', df_goibibo['hotel_description'].isna().sum())
print('address: ', df_goibibo['address'].isna().sum())
print('city: ', df_goibibo['city'].isna().sum())
print('state: ', df_goibibo['state'].isna().sum())
print('country: ', df_goibibo['country'].isna().sum())
print('latitude: ', df_goibibo['latitude'].isna().sum())
print('longitude: ', df_goibibo['longitude'].isna().sum())
print('point_of_interest: ', df_goibibo['point_of_interest'].isna().sum())
print('hotel_facilities: ', df_goibibo['hotel_facilities'].isna().sum())
print('hotel_star_rating: ', df_goibibo['hotel_star_rating'].isna().sum())
print('room_facilities: ', df_goibibo['room_facilities'].isna().sum())
# print('room_type: ', df_goibibo['room_type'].isna().sum())

print('site_review_rating: ', df_goibibo['site_review_rating'].isna().sum())

cek jumlah null dari tiap parameter: 

property_name:  0
property_type:  0
property_id:  0
hotel_description:  0
address:  0
city:  0
state:  0
country:  0
latitude:  0
longitude:  0
point_of_interest:  0
hotel_facilities:  0
hotel_star_rating:  0
room_facilities:  0
site_review_rating:  0


In [28]:
print('Daftar tipe properti: \n', len(df_goibibo['property_type'].unique()), df_goibibo['property_type'].unique())
print('Total kota: ',len(df_goibibo['city'].unique()))
print('Total provinsi: ',len(df_goibibo['state'].unique()), df_goibibo['state'].unique())
print('Daftar hotel rating: ',len(df_goibibo['hotel_star_rating'].unique()), df_goibibo['hotel_star_rating'].unique())

# room type terlalu beragam, tidak standar jenisnya
# print('Daftar Room Type: ',len(df_goibibo['room_type'].unique()))

Daftar tipe properti: 
 17 ['Resort' 'Guest House' 'Cottage' 'Hotel' 'Homestay' 'Palace'
 'Service Apartment' 'Hostel' 'Lodge' 'Tent' 'Villa' 'Houseboat'
 'Bungalow' 'BnB' 'Beach Hut' 'Motel' 'Farm Stay']
Total kota:  330
Total provinsi:  27 ['Himachal Pradesh' 'Haryana' 'Goa' 'Delhi' 'Rajasthan' 'Assam'
 'Tamil Nadu' 'Gujarat' 'West Bengal' 'Maharashtra' 'Karnataka' 'Kerala'
 'Punjab' 'Uttarakhand' 'Madhya Pradesh' 'Uttar Pradesh' 'Sikkim'
 'Andaman and Nicobar Islands' 'Andhra Pradesh' 'Orissa'
 'Jammu and Kashmir' 'Jharkhand' 'Pondicherry' 'Bihar' 'Meghalaya'
 'Dadra and Nagar Haveli' 'Chhattisgarh']
Daftar hotel rating:  6 [2 0 3 1 4 5]


In [29]:
# melihat tipe data
df_goibibo.dtypes

address                object
city                   object
country                object
hotel_description      object
hotel_facilities       object
hotel_star_rating       int64
latitude              float64
longitude             float64
point_of_interest      object
property_id            object
property_name          object
property_type          object
room_facilities        object
site_review_rating    float64
state                  object
dtype: object

In [30]:
# change nested data to array
df_goibibo['hotel_facilities'] = [item.split("|") for item in df_goibibo['hotel_facilities']]
df_goibibo['point_of_interest'] = [item.split("|") for item in df_goibibo['point_of_interest']]
df_goibibo['room_facilities'] = [item.split("|") for item in df_goibibo['room_facilities']]
df_goibibo

,address,city,country,hotel_description,hotel_facilities,hotel_star_rating,latitude,longitude,point_of_interest,property_id,property_name,property_type,room_facilities,site_review_rating,state
0,"15th Mile, N.H.21,Manali, District Kullu,Himac...",Manali,India,The standard check-in time is 12:00 PM and the...,"[Doctor on Call, Dry Cleaning, Laundry Service...",2,32.139387,77.154660,"[Hadimba Temple, Naggar Village, Himalayan Nyi...",HTLGBO1000016238,Baragarh Regency,Resort,"[Room Service , Basic Bathroom Amenities, Cabl...",4.0,Himachal Pradesh
1,"A-585, Sushant Lok-1 ,Near Iffco Chowk Metro S...",Gurgaon,India,The standard check-in time is 12:00 PM and the...,"[Airport Transfer Available / Surcharge, Banqu...",0,28.472097,77.072546,"[Sushant Lok, Sahara Mall, Amity International...",HTLGBO1000015822,Asian Suites A- 585,Guest House,"[Room Service , Air Conditioning , Basic Bathr...",4.5,Haryana
2,"Cobra Vaddo,Calungate Baga Road, Bardez, Calan...",Goa,India,The standard check-in time is 12:00 PM and the...,"[Swimming Pool, Bar / Lounge , Laundry Service...",0,15.548398,73.757634,"[Anjuna Beach, Calangute Beach, Titos lane, ba...",HTLGBO1000004551,Bevvan Resort,Resort,"[Room Service , Air Conditioning , Cable / Sat...",2.5,Goa
3,Simsa,Manali,India,The standard check-in time is 12:00 PM and the...,"[Doctor on Call, Dry Cleaning, Laundry Service...",2,32.223603,77.185900,"[The Mall, Tibetian Monastery, Vashisht Hot Ba...",HTLGBO1000007993,Apple Inn Cottage,Cottage,"[Basic Bathroom Amenities, Cable / Satellite /...",5.0,Himachal Pradesh
4,"8180 Street No.-6,Arakashan Road,Paharganj",Delhi,India,The standard check-in time is 12:00 PM and the...,"[Internet Access - Surcharge, Laundry Service ...",2,28.646777,77.212735,"[Gaffar Market, YMCA Institute, Agrasen Boali,...",HTLGBO1000008281,Anmol Hotel Pvt.Ltd,Hotel,"[Basic Bathroom Amenities, Cable / Satellite /...",2.8,Delhi
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3992,"sarvaritu vilas , inside udaipole , opp sagasj...",Udaipur,India,The standard check-in time is 12:00 PM and the...,"[Airport Transfer Available / Surcharge, Docto...",2,24.575598,73.696959,"[Jagdish Temple, City Palace, Udaipur Bus Stan...",HTLGBO1000012898,Hotel Shubh Laxmi,Hotel,"[Room Service , Basic Bathroom Amenities, Cabl...",4.2,Rajasthan
3993,"Near Yasholaxmi Nagar, Kolhapur Rd,",Ichalkaranji,India,The standard check-in time is 10:00 AM and the...,"[Bar / Lounge , Doctor on Call, Dry Cleaning, ...",2,16.690042,74.456394,[Yasholaxmi Nagar],HTLGBO1000083318,Hotel Signature,Hotel,"[Room Service , Air Conditioning , Alarm Clock...",5.0,Maharashtra
3994,"Near Railway Station , Kota Jn",Kota,India,The standard check-in time is 12:00 PM and the...,"[Bar / Lounge , Doctor on Call, Laundry Servic...",2,25.222694,75.878170,"[Kota Airport, Station Road Area]",HTLGBO1000009343,Hotel Shree Anand Restaurant &amp; Bar,Hotel,"[Room Service , Air Conditioning , Basic Bathr...",3.4,Rajasthan
3997,"Marol Maroshi Road, Near To Marol naka FireBri...",Mumbai,India,The standard check-in time is 12:00 PM and the...,"[Banquet Facilities, Bar / Lounge , Currency E...",3,19.110926,72.879038,"[Mount Mary Church, Juhu Beach, Domestic Airpo...",HTLGBO1000001584,Hotel Silver Inn,Hotel,"[Air Conditioning , Cable / Satellite / Pay TV...",4.0,Maharashtra


In [31]:
df_goibibo[['hotel_facilities','room_facilities','point_of_interest']]

,hotel_facilities,room_facilities,point_of_interest
0,"[Doctor on Call, Dry Cleaning, Laundry Service...","[Room Service , Basic Bathroom Amenities, Cabl...","[Hadimba Temple, Naggar Village, Himalayan Nyi..."
1,"[Airport Transfer Available / Surcharge, Banqu...","[Room Service , Air Conditioning , Basic Bathr...","[Sushant Lok, Sahara Mall, Amity International..."
2,"[Swimming Pool, Bar / Lounge , Laundry Service...","[Room Service , Air Conditioning , Cable / Sat...","[Anjuna Beach, Calangute Beach, Titos lane, ba..."
3,"[Doctor on Call, Dry Cleaning, Laundry Service...","[Basic Bathroom Amenities, Cable / Satellite /...","[The Mall, Tibetian Monastery, Vashisht Hot Ba..."
4,"[Internet Access - Surcharge, Laundry Service ...","[Basic Bathroom Amenities, Cable / Satellite /...","[Gaffar Market, YMCA Institute, Agrasen Boali,..."
...,...,...,...
3992,"[Airport Transfer Available / Surcharge, Docto...","[Room Service , Basic Bathroom Amenities, Cabl...","[Jagdish Temple, City Palace, Udaipur Bus Stan..."
3993,"[Bar / Lounge , Doctor on Call, Dry Cleaning, ...","[Room Service , Air Conditioning , Alarm Clock...",[Yasholaxmi Nagar]
3994,"[Bar / Lounge , Doctor on Call, Laundry Servic...","[Room Service , Air Conditioning , Basic Bathr...","[Kota Airport, Station Road Area]"
3997,"[Banquet Facilities, Bar / Lounge , Currency E...","[Air Conditioning , Cable / Satellite / Pay TV...","[Mount Mary Church, Juhu Beach, Domestic Airpo..."


In [34]:
# rearrange the feature position
df_goibibo = df_goibibo[['property_id','property_name','hotel_description','address','point_of_interest','city','state','country','latitude','longitude','property_type','hotel_star_rating','hotel_facilities','room_facilities']]

# recreate the property_id for better
hid = []
for n in range(1, len(df_goibibo)+1):
    hid.append('H{0:06}'.format(n))
df_goibibo['property_id'] = hid
df_goibibo.to_csv('DataHotel_nonML.csv')

In [35]:
df_goibibo.head(3)

,property_id,property_name,hotel_description,address,point_of_interest,city,state,country,latitude,longitude,property_type,hotel_star_rating,hotel_facilities,room_facilities
0,H000001,Baragarh Regency,The standard check-in time is 12:00 PM and the...,"15th Mile, N.H.21,Manali, District Kullu,Himac...","[Hadimba Temple, Naggar Village, Himalayan Nyi...",Manali,Himachal Pradesh,India,32.139387,77.154660,Resort,2,"[Doctor on Call, Dry Cleaning, Laundry Service...","[Room Service , Basic Bathroom Amenities, Cabl..."
1,H000002,Asian Suites A- 585,The standard check-in time is 12:00 PM and the...,"A-585, Sushant Lok-1 ,Near Iffco Chowk Metro S...","[Sushant Lok, Sahara Mall, Amity International...",Gurgaon,Haryana,India,28.472097,77.072546,Guest House,0,"[Airport Transfer Available / Surcharge, Banqu...","[Room Service , Air Conditioning , Basic Bathr..."
2,H000003,Bevvan Resort,The standard check-in time is 12:00 PM and the...,"Cobra Vaddo,Calungate Baga Road, Bardez, Calan...","[Anjuna Beach, Calangute Beach, Titos lane, ba...",Goa,Goa,India,15.548398,73.757634,Resort,0,"[Swimming Pool, Bar / Lounge , Laundry Service...","[Room Service , Air Conditioning , Cable / Sat..."


In [36]:
hotel = set()
for item in df_goibibo['hotel_facilities']:
    for h in item:
        hotel.add(h)
room = set()
for item in df_goibibo['room_facilities']:
    for r in item:
        room.add(r)
poi = set()
for item in df_goibibo['point_of_interest']:
    for p in item:
        poi.add(p)

In [37]:
print('Jumlah unique hotel facilities: ', len(hotel))
print('Jumlah unique room facilities: ', len(room))
print('Jumlah unique point of interest: ', len(poi))
# POI Terlalu beragam dan tidak standar sehingga tidak diperhitungkan dahulu dalam ML

Jumlah unique hotel facilities:  202
Jumlah unique room facilities:  166
Jumlah unique point of interest:  4218
